In [6]:


import sys
import pickle
from pprint import pprint as pp
import numpy as np
#np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import GenericUnivariateSelect, chi2, f_classif
from sklearn import tree
from sklearn.metrics import accuracy_score

#sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data








### Task 1: Select what features you'll use (& pre analysis via Correlation Matrix).
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

features_list = [
    'poi',
    'salary',
    'to_messages',
    'deferral_payments',
    'total_payments',
    'exercised_stock_options',
    'bonus',
    'restricted_stock',
    'shared_receipt_with_poi',
    'restricted_stock_deferred',
    'total_stock_value',
    'expenses',
    'loan_advances',
    'from_messages',
    'other',
    'from_this_person_to_poi',
    'director_fees',
    'deferred_income',
    'long_term_incentive',
    #'email_address',
    'from_poi_to_this_person',
]

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)


#basic function to replace NaN values
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False
for _, values in data_dict.items():
    for key, value in values.items():
        if isnan(value):
            values[key] = 0

#Correlation Matrix with Heatmap (from https://towardsdatascience.com/)
pd_data = pd.DataFrame.from_dict(data_dict, orient='index')

corrmat = pd_data.corr()
top_corr_features = corrmat.index
#plt.figure(figsize=(20,20))
#g=sns.heatmap(pd_data[top_corr_features].corr(),annot=True,cmap="RdYlGn")








### Task 2: Remove outliers
#Compte avant
poi = set()
for empl in data_dict:
    if data_dict[empl]['poi'] == True:poi.add(empl)

#print "nb of poi = {}, nb of empl. = {}. Ratio = {}%".format(len(poi), len(data_dict),\
#                                                            round( float( len(poi)) / len(data_dict), 4)*100 )

# Computing IQR using Pandas functionnality
Q1 = pd_data.quantile(0.25)
Q3 = pd_data.quantile(0.75)
IQR = Q3 - Q1
series_outl = [pd_data[(pd_data>(Q3 + 1.5*IQR) ) | (pd_data<(Q1 - 1.5*IQR) )].count(axis=1), pd_data['poi']]
series_outl[0].name = "bad_feats"
outl = pd.concat(series_outl, axis=1)

outl.sort_values(by='bad_feats', ascending=False, inplace=True)
#print outl.head(8)

#Removing outliers
outliers = ('LAY KENNETH L', 'TOTAL', 'FREVERT MARK A', 'BELDEN TIMOTHY N', 'BAXTER JOHN C',\
            'SKILLING JEFFREY K', 'LAVORATO JOHN J', 'HAEDICKE MARK E')
for k in outliers:
    data_dict.pop(k, None)


#Compte après
poi2 = set()
for empl in data_dict:
    if data_dict[empl]['poi'] == True:poi2.add(empl)

#print "nb of poi = {}, nb of empl. = {}. Ratio = {}%".format(len(poi2), len(data_dict),\
#                                                            round( float( len(poi2)) / len(data_dict), 4)*100 )








### Task 2 bis: Cleaning data

#Check sums
payment_sum = [
    'salary',
    'deferral_payments',
    'bonus',
    'expenses',
    'loan_advances',
    'other',
    'director_fees',
    'deferred_income',
    'long_term_incentive',
]

stock_sum = [
    'exercised_stock_options',
    'restricted_stock',
    'restricted_stock_deferred',
]

delta_pay = pd_data[payment_sum].sum(axis=1) != pd_data['total_payments']
delta_stock = pd_data[stock_sum].sum(axis=1) != pd_data['total_stock_value']
#print list(delta_pay[delta_pay==True].index), list(delta_pay[delta_stock==True].index)


#Corrections
data_dict['BELFER ROBERT']['deferral_payments'] = 0
data_dict['BELFER ROBERT']['total_payments'] = 3285
data_dict['BELFER ROBERT']['exercised_stock_options'] = 0
data_dict['BELFER ROBERT']['restricted_stock'] = 44093
data_dict['BELFER ROBERT']['restricted_stock_deferred'] = -44093
data_dict['BELFER ROBERT']['total_stock_value'] = 0
data_dict['BELFER ROBERT']['expenses'] = 3285
data_dict['BELFER ROBERT']['deferred_income'] = -102500
data_dict['BELFER ROBERT']['director_fees'] = 102500

data_dict['BHATNAGAR SANJAY']['total_payments'] = 137864
data_dict['BHATNAGAR SANJAY']['exercised_stock_options'] = 15456290
data_dict['BHATNAGAR SANJAY']['restricted_stock'] = 2604490
data_dict['BHATNAGAR SANJAY']['restricted_stock_deferred'] = -2604490
data_dict['BHATNAGAR SANJAY']['total_stock_value'] = 15456290
data_dict['BHATNAGAR SANJAY']['expenses'] = 137864
data_dict['BHATNAGAR SANJAY']['other'] = 0
data_dict['BHATNAGAR SANJAY']['director_fees'] = 0

pd_data_corr = pd.DataFrame.from_dict(data_dict, orient='index')

delta_pay = pd_data_corr[payment_sum].sum(axis=1) != pd_data_corr['total_payments']
delta_stock = pd_data_corr[stock_sum].sum(axis=1) != pd_data_corr['total_stock_value']

#print list(delta_pay[delta_pay==True].index), list(delta_pay[delta_stock==True].index)












### Task 3: Create new feature(s)


def computeFraction( poi_messages, all_messages ):
    """ given a number messages to/from POI (numerator) 
        and number of all messages to/from a person (denominator),
        return the fraction of messages to/from that person
        that are from/to a POI
    """
    
    fraction = 0.
    
    poi_messages = float(poi_messages)
    all_messages = float(all_messages)
    
    if isnan(poi_messages) or poi_messages == 0:
        fraction = 0.
    else: fraction = poi_messages / all_messages
    
    return round(fraction, 3)


submit_dict = {}
for name in data_dict:

    data_point = data_dict[name]
    
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
    data_point["fraction_from_poi"] = fraction_from_poi

    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
    submit_dict[name]={"from_poi_to_this_person":fraction_from_poi,
                       "from_this_person_to_poi":fraction_to_poi}
    data_point["fraction_to_poi"] = fraction_to_poi

pp(data_dict)








### Task 3 bis: Scaling















### Store to my_dataset for easy export below.
my_dataset = data_dict



### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:16.0f}'.format}, linewidth=130)
#pp(data)




#Univariate feature selector with configurable strategy
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html

"""
print len(features[0])
transformer = GenericUnivariateSelect(f_classif, mode='k_best', param=5) #['fwe', 'percentile', 'fdr', 'k_best', 'fpr']
features_uni = transformer.fit_transform(features, labels)
print len(features_uni[0])
"""











### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)

{'ALLEN PHILLIP K': {'bonus': 4175000,
                     'deferral_payments': 2869717,
                     'deferred_income': -3081055,
                     'director_fees': 0,
                     'email_address': 'phillip.allen@enron.com',
                     'exercised_stock_options': 1729541,
                     'expenses': 13868,
                     'fraction_from_poi': 0.016,
                     'fraction_to_poi': 0.03,
                     'from_messages': 2195,
                     'from_poi_to_this_person': 47,
                     'from_this_person_to_poi': 65,
                     'loan_advances': 0,
                     'long_term_incentive': 304805,
                     'other': 152,
                     'poi': False,
                     'restricted_stock': 126027,
                     'restricted_stock_deferred': -126027,
                     'salary': 201955,
                     'shared_receipt_with_poi': 1407,
                     'to_messages': 2902,
        

 'GILLIS JOHN': {'bonus': 0,
                 'deferral_payments': 0,
                 'deferred_income': 0,
                 'director_fees': 0,
                 'email_address': 0,
                 'exercised_stock_options': 9803,
                 'expenses': 0,
                 'fraction_from_poi': 0.0,
                 'fraction_to_poi': 0.0,
                 'from_messages': 0,
                 'from_poi_to_this_person': 0,
                 'from_this_person_to_poi': 0,
                 'loan_advances': 0,
                 'long_term_incentive': 0,
                 'other': 0,
                 'poi': False,
                 'restricted_stock': 75838,
                 'restricted_stock_deferred': 0,
                 'salary': 0,
                 'shared_receipt_with_poi': 0,
                 'to_messages': 0,
                 'total_payments': 0,
                 'total_stock_value': 85641},
 'GLISAN JR BEN F': {'bonus': 600000,
                     'deferral_payments': 0,
        

                'to_messages': 807,
                'total_payments': 1061827,
                'total_stock_value': 585062},
 'MEYER JEROME J': {'bonus': 0,
                    'deferral_payments': 0,
                    'deferred_income': -38346,
                    'director_fees': 38346,
                    'email_address': 0,
                    'exercised_stock_options': 0,
                    'expenses': 2151,
                    'fraction_from_poi': 0.0,
                    'fraction_to_poi': 0.0,
                    'from_messages': 0,
                    'from_poi_to_this_person': 0,
                    'from_this_person_to_poi': 0,
                    'loan_advances': 0,
                    'long_term_incentive': 0,
                    'other': 0,
                    'poi': False,
                    'restricted_stock': 0,
                    'restricted_stock_deferred': 0,
                    'salary': 0,
                    'shared_receipt_with_poi': 0,
                    

                        'shared_receipt_with_poi': 0,
                        'to_messages': 0,
                        'total_payments': 762135,
                        'total_stock_value': 384930},
 'WHALEY DAVID A': {'bonus': 0,
                    'deferral_payments': 0,
                    'deferred_income': 0,
                    'director_fees': 0,
                    'email_address': 0,
                    'exercised_stock_options': 98718,
                    'expenses': 0,
                    'fraction_from_poi': 0.0,
                    'fraction_to_poi': 0.0,
                    'from_messages': 0,
                    'from_poi_to_this_person': 0,
                    'from_this_person_to_poi': 0,
                    'loan_advances': 0,
                    'long_term_incentive': 0,
                    'other': 0,
                    'poi': False,
                    'restricted_stock': 0,
                    'restricted_stock_deferred': 0,
                    'salary': 0,
   